# Поиск наиболее близкого имени участника

In [1]:
# conda install -c conda-forge python-levenshtein
# или pip install python-levenshtein
import numpy as np
import Levenshtein
import pandas as pd

In [102]:
# что было написано в заявке

claim_list = [
    'Шехавцова Анна',
    'Гречихина Наталья',
    'Козлова Алена',
    'Груздева Алена',
    'Груздева Полина',
    'Кущенко Анна',
    'Чистякова Анастасия'
]

In [103]:
# результат расшифровки речи диктора

speech_recognition = [
    'кучменко она',
    'кущенко оксана',
    'груздь алина',
    'рычихина наталья',
    'шиховцева на',
    'чистова анастасия'
]

### С библиотекой Levenshtein

In [104]:
Levenshtein.distance('Оксана', 'Анна')

4

In [105]:
for recognized_name in speech_recognition:
    print(recognized_name)

кучменко она
кущенко оксана
груздь алина
рычихина наталья
шиховцева на
чистова анастасия


In [106]:
for recognized_name in speech_recognition:
    for real_name in claim_list:
        print('{},{},{}'.format(recognized_name, real_name, Levenshtein.distance(recognized_name, real_name)))

кучменко она,Шехавцова Анна,11
кучменко она,Гречихина Наталья,14
кучменко она,Козлова Алена,11
кучменко она,Груздева Алена,10
кучменко она,Груздева Полина,10
кучменко она,Кущенко Анна,5
кучменко она,Чистякова Анастасия,15
кущенко оксана,Шехавцова Анна,11
кущенко оксана,Гречихина Наталья,15
кущенко оксана,Козлова Алена,11
кущенко оксана,Груздева Алена,11
кущенко оксана,Груздева Полина,11
кущенко оксана,Кущенко Анна,5
кущенко оксана,Чистякова Анастасия,14
груздь алина,Шехавцова Анна,11
груздь алина,Гречихина Наталья,14
груздь алина,Козлова Алена,9
груздь алина,Груздева Алена,6
груздь алина,Груздева Полина,6
груздь алина,Кущенко Анна,10
груздь алина,Чистякова Анастасия,16
рычихина наталья,Шехавцова Анна,14
рычихина наталья,Гречихина Наталья,3
рычихина наталья,Козлова Алена,14
рычихина наталья,Груздева Алена,14
рычихина наталья,Груздева Полина,14
рычихина наталья,Кущенко Анна,14
рычихина наталья,Чистякова Анастасия,12
шиховцева на,Шехавцова Анна,6
шиховцева на,Гречихина Наталья,14
шиховцев

### Без библиотеки из файла 'recognized_stats.txt'

In [107]:
# distances = []

# f = open('recognized_stats.txt', mode = 'r', encoding = 'utf-8')

# lines = [line.strip().split(';') for line in f]

# for line in lines:
    
#     for rec in line:
#         recognized_name, real_name, distance = rec.split(',')
#         print(recognized_name, real_name, distance)

### Сортировка словаря

In [108]:
for recognized_name in speech_recognition:
    distances = {real_name: Levenshtein.distance(recognized_name, real_name) for real_name in claim_list}
    print(distances)
    
    break

{'Шехавцова Анна': 11, 'Гречихина Наталья': 14, 'Козлова Алена': 11, 'Груздева Алена': 10, 'Груздева Полина': 10, 'Кущенко Анна': 5, 'Чистякова Анастасия': 15}


In [109]:
# x[1] - сортировка по значениям словаря по возрастанию (если надо по убыванию - добавляем минус перед x[1])
# x[0] - по ключам
sorted(distances.items(), key = lambda x: x[1])

[('Кущенко Анна', 5),
 ('Груздева Алена', 10),
 ('Груздева Полина', 10),
 ('Шехавцова Анна', 11),
 ('Козлова Алена', 11),
 ('Гречихина Наталья', 14),
 ('Чистякова Анастасия', 15)]

In [110]:
def find_match(speech_recognition, claim_list):
    '''
    Функция принимает список распознанных и реальных имен.
    Рассчитывает расстояние Левенштейна между всеми парами
    распознанных и реальных имен и возвращает DataFrame
    соответствия распознанного имени реальному имени (или именам).
    '''
    fio_match = {}
    min_distances = []
    claim_list_arr = np.array(claim_list)
    for recognized_name in speech_recognition:
        distances = np.array([Levenshtein.distance(recognized_name, name) for name in claim_list])
        if (distances==distances.min()).sum() == 1:
            fio_match[recognized_name] = claim_list[distances.argmin()]
        else:
            fio_match[recognized_name] = claim_list_arr[np.where(distances==distances.min())[0]]       
        min_distances.append(distances.min())
    match_df = pd.DataFrame({'Распознанное имя': list(fio_match.keys()), 
                            'Реальное имя': list(fio_match.values()),
                            'Расстояние Левенштейна':  min_distances})
    return match_df

In [111]:
find_match(speech_recognition, claim_list)

,Распознанное имя,Реальное имя,Расстояние Левенштейна
0,кучменко она,Кущенко Анна,5
1,кущенко оксана,Кущенко Анна,5
2,груздь алина,"[Груздева Алена, Груздева Полина]",6
3,рычихина наталья,Гречихина Наталья,3
4,шиховцева на,Шехавцова Анна,6
5,чистова анастасия,Чистякова Анастасия,4
